# Cypher queries

## Testing import from blocks and transaction topics by sample queries

In [15]:
import numpy as np
import time
import sys
sys.path.append("./evaluation")
from neo4jConnector import startNeo4jSession

## Conversion functions 

In [3]:
#### Convert final query by cutting all linebreaks...
def createCypherForEvents(query):
    new_query = query.replace('$', 'event.')
    return new_query

#### Convert final query by cutting all linebreaks...
def omitSpacings(query):
    omit_characters = ["\n", "     ", "     ", "  "]
    for i in omit_characters:
        query = query.replace(i, " ")
    return query

## Transactions


In [19]:
transactionQuery = '''
    MERGE (t:Transaction{txid: $txid}) 
        SET t += {inDegree: $inDegree, outDegree: $outDegree, outSum: $outSum, inSum: $inSum, date: date($block_date)}

    MERGE (b:Block{hash: $block_hash})
    MERGE (t)-[:BELONGS_TO]->(b)
    
    FOREACH (output in $output_list | 
        MERGE (o_a:Address{address: output.addr}) 
            ON CREATE SET o_a += {inDegree: 1, outDegree: 0}
            ON MATCH  SET o_a += {inDegree: o_a.inDegree + 1}
        MERGE (t)-[r:RECEIVES{output_nr: output.outNr, value: output.val}]->(o_a))
        
    FOREACH (input in $input_list | 
        MERGE (i_a:Address{address: input.addr})
            ON CREATE SET i_a += {inDegree: 0, outDegree: 1}
            ON MATCH SET  i_a += {outDegree: i_a.outDegree + 1}
        MERGE (i_a)-[:SENDS{value: input.val}]->(t))
    '''


## amanded for test and creationTime

transactionQuery = '''
    MERGE (t:Transaction_test{txid: $txid}) 
        SET t += {creationTime:apoc.date.currentTimestamp(), inDegree: $inDegree, outDegree: $outDegree, outSum: $outSum, inSum: $inSum, date: date($block_date)}

    MERGE (b:Block_test{hash: $block_hash})
    MERGE (t)-[:BELONGS_TO]->(b)
    
    FOREACH (output in $output_list | 
        MERGE (o_a:Address_test{address: output.addr}) 
            ON CREATE SET o_a += {creationTime:apoc.date.currentTimestamp(), inDegree: 1, outDegree: 0}
            ON MATCH  SET o_a += {inDegree: o_a.inDegree + 1}
        MERGE (t)-[r:RECEIVES{output_nr: output.outNr, value: output.val}]->(o_a))
        
    FOREACH (input in $input_list | 
        MERGE (i_a:Address_test{address: input.addr})
            ON CREATE SET i_a += {creationTime:apoc.date.currentTimestamp(), inDegree: 0, outDegree: 1}
            ON MATCH SET  i_a += {outDegree: i_a.outDegree + 1}
        MERGE (i_a)-[:SENDS{value: input.val}]->(t))
    '''



In [1]:
## batched transactions

transactionQuery = '''
FOREACH (event in $event |
    MERGE (t:Transaction_test{txid: $txid}) 
        SET t += {creationTime:apoc.date.currentTimestamp(), inDegree: $inDegree, outDegree: $outDegree, outSum: $outSum, inSum: $inSum, date: date($block_date)}

    MERGE (b:Block_test{hash: $block_hash})
    MERGE (t)-[:BELONGS_TO]->(b)
    
    FOREACH (output in $output_list | 
        MERGE (o_a:Address_test{address: output.addr}) 
            ON CREATE SET o_a += {creationTime:apoc.date.currentTimestamp(), inDegree: 1, outDegree: 0}
            ON MATCH  SET o_a += {inDegree: o_a.inDegree + 1}
        MERGE (t)-[r:RECEIVES{output_nr: output.outNr, value: output.val}]->(o_a))
        
    FOREACH (input in $input_list | 
        MERGE (i_a:Address_test{address: input.addr})
            ON CREATE SET i_a += {creationTime:apoc.date.currentTimestamp(), inDegree: 0, outDegree: 1}
            ON MATCH SET  i_a += {outDegree: i_a.outDegree + 1}
        MERGE (i_a)-[:SENDS{value: input.val}]->(t))
    )
'''

## Final TX query

In [4]:
#### Convert final query by cutting all linebreaks...
transactionQuery_forEvents = createCypherForEvents(transactionQuery)
print(transactionQuery_forEvents)


FOREACH (event in event.event |
    MERGE (t:Transaction_test{txid: event.txid}) 
        SET t += {creationTime:apoc.date.currentTimestamp(), inDegree: event.inDegree, outDegree: event.outDegree, outSum: event.outSum, inSum: event.inSum, date: date(event.block_date)}

    MERGE (b:Block_test{hash: event.block_hash})
    MERGE (t)-[:BELONGS_TO]->(b)
    
    FOREACH (output in event.output_list | 
        MERGE (o_a:Address_test{address: output.addr}) 
            ON CREATE SET o_a += {creationTime:apoc.date.currentTimestamp(), inDegree: 1, outDegree: 0}
            ON MATCH  SET o_a += {inDegree: o_a.inDegree + 1}
        MERGE (t)-[r:RECEIVES{output_nr: output.outNr, value: output.val}]->(o_a))
        
    FOREACH (input in event.input_list | 
        MERGE (i_a:Address_test{address: input.addr})
            ON CREATE SET i_a += {creationTime:apoc.date.currentTimestamp(), inDegree: 0, outDegree: 1}
            ON MATCH SET  i_a += {outDegree: i_a.outDegree + 1}
        MERGE (i_a)-

In [5]:
#### Convert final query by cutting all linebreaks...
transactionQuery_final = omitSpacings(transactionQuery_forEvents)
print(transactionQuery_final)

 FOREACH (event in event.event | MERGE (t:Transaction_test{txid: event.txid}) SET t += {creationTime:apoc.date.currentTimestamp(), inDegree: event.inDegree, outDegree: event.outDegree, outSum: event.outSum, inSum: event.inSum, date: date(event.block_date)} MERGE (b:Block_test{hash: event.block_hash}) MERGE (t)-[:BELONGS_TO]->(b) FOREACH (output in event.output_list | MERGE (o_a:Address_test{address: output.addr}) ON CREATE SET o_a += {creationTime:apoc.date.currentTimestamp(), inDegree: 1, outDegree: 0} ON MATCH SET o_a += {inDegree: o_a.inDegree + 1} MERGE (t)-[r:RECEIVES{output_nr: output.outNr, value: output.val}]->(o_a)) FOREACH (input in event.input_list | MERGE (i_a:Address_test{address: input.addr}) ON CREATE SET i_a += {creationTime:apoc.date.currentTimestamp(), inDegree: 0, outDegree: 1} ON MATCH SET i_a += {outDegree: i_a.outDegree + 1} MERGE (i_a)-[:SENDS{value: input.val}]->(t)) ) 


## Blocks

In [22]:
## !need to amend!
blockQuery = '''
MERGE (p:Block_test {hash: $previousblockhash})
MERGE (b:Block_test {hash: $block_hash}) 
    SET b += {creationTime:apoc.date.currentTimestamp(), hash: $block_hash, height: $block_height, blockDate: date($block_date), mediantime: datetime($block_median_time) }
MERGE (p)-[r:PRECEDES]->(b)
'''

## Final Block query

In [23]:
#### Convert final query by cutting all linebreaks...
blockQuery_forEvents = createCypherForEvents(blockQuery)
print(blockQuery_forEvents)


MERGE (p:Block_test {hash: event.previousblockhash})
MERGE (b:Block_test {hash: event.block_hash}) 
    SET b += {creationTime:apoc.date.currentTimestamp(), hash: event.block_hash, height: event.block_height, blockDate: date(event.block_date), mediantime: datetime(event.block_median_time) }
MERGE (p)-[r:PRECEDES]->(b)



In [24]:
#### Convert final query by cutting all linebreaks...
blockQuery_final = omitSpacings(blockQuery_forEvents)
print(blockQuery_final)

 MERGE (p:Block_test {hash: event.previousblockhash}) MERGE (b:Block_test {hash: event.block_hash}) SET b += {creationTime:apoc.date.currentTimestamp(), hash: event.block_hash, height: event.block_height, blockDate: date(event.block_date), mediantime: datetime(event.block_median_time) } MERGE (p)-[r:PRECEDES]->(b) 


## Querying Neo4j directly

In [25]:
#driver, session = startNeo4jSession(port=7688)

driver, session = startNeo4jSession(location = 'local', port=7687)

### Example Data

In [28]:
#test data 
test = {'txid': '88d35b8d35ccaf8a315a2274d83081ab62aeda06fb2029af5c65b2a858f22fcb',
 'block_hash': '00000000000000000005278b3b0d99196ded30c92edb64181abab711fb47527b',
 'block_date': '2021-05-23',
 'outDegree': 3,
 'inDegree': 1,
 'outSum': 656215465,
 'inSum': 656215465,
 'input_list': [{'addr': 'coinbase', 'val': 656215465}],
 'output_list': [{'addr': '1Bf9sZvBHPFGVPX71WX2njhd1NXKv5y7v5', 'outNr': 0, 'val': 5}]}

test = {'txid': 'd88fbb53146b8662b9e3d58f245d5d7ae0a841e8eb1ad13e8e3e8504a0d8fcea', 'block_hash': '000000000000098a0adb1934bee96bf43430257c9ed8759a219e45bf5758785c', 'block_date': '2012-01-30', 'outDegree': 2, 'inDegree': 1, 'outSum': 1089650000, 'inSum': 1089700000, 'input_list': [{'addr': '1KTWQ7aKeNyGKRtqCrkDTtrbWJphcsZMe3', 'val': 1089700000}], 'output_list': [{'addr': '1GFJf5o9DSjrN8V5VQnMqEfCR8Wa6QXVZn', 'val': 1089150000, 'outNr': 0}, {'addr': '1KNYrxYNpfCzoYZLYRSj9mFkX9M7xDUWpF', 'val': 500000, 'outNr': 1}]}
test = {'txid': '0e3e2357e806b6cdb1f70b54c3a3a17b6714ee1f0e68bebb44a74b1efd512098', 'block_hash': '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048', 'block_date': '2009-01-09', 'outDegree': 1, 'inDegree': 1, 'outSum': 5000000000, 'inSum': 5000000000, 'input_list': [{'addr': 'coinbase', 'val': 5000000000}], 'output_list': [{'addr': '12c6DSiU4Rq3P4ZxziKxzrL5LmMBrzjrJX', 'val': 5000000000, 'outNr': 0}]}

block_data = {'block_hash': '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048', 
              'block_height': 1, 
              'block_median_time': '2009-01-09T03:54', 
              'block_date': '2009-01-09', 
              'previousblockhash': '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f'}

### Transactions

In [29]:
def run_tx(tx, event):
    tx.run(transactionQuery, 
           txid         = event["txid"], 
           output_list  = event["output_list"], 
           block_hash   = event["block_hash"], 
           inDegree     = event["inDegree"], 
           outDegree    = event["outDegree"], 
           outSum       = event["outSum"],
           inSum        = event["inSum"],
           input_list   = event["input_list"], 
           block_date   = event["block_date"])


with driver.session() as session:
    session.write_transaction(run_tx, test)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Unknown function 'apoc.date.currentTimestamp' (line 3, column 36 (offset: 77))
"SET t += {creationTime: apoc.date.currentTimestamp(), inDegree: $inDegree, outDegree: $outDegree, outSum: $outSum, inSum: $inSum, date: date($block_date)}"
                                     ^}

### Blocks

In [ ]:
def create_block(tx, block_hash, block_height, block_median_time, block_date, previousblockhash):
    tx.run(blockQuery, 
           block_hash=block_hash, 
           block_height=block_height, 
           block_median_time=block_median_time, 
           block_date=block_date,
           previousblockhash=previousblockhash)

block = block_data
with driver.session() as session:
    session.write_transaction(create_block, 
                              block['block_hash'], 
                              block['block_height'], 
                              block['block_median_time'],
                              block['block_date'],
                              block['previousblockhash'])
        